In [ ]:
import os
from pathlib import Path
from PIL import Image

In [ ]:
from source.visual_genome_meta_data import read_json_to_dict
from source.visual_genome_meta_data import get_image_meta_data
from source.visual_genome_meta_data import count_occurrences
from source.visual_genome_to_yolo import create_class_mapping_from_list
from source.visual_genome_to_yolo import save_class_map_to_yaml
from source.visual_genome_to_yolo import convert_single_image_to_yolo
from source.visual_genome_to_yolo import read_yaml_to_class_map
from source.visual_genome_to_yolo import read_yolo_metadata
# from source.visual_genome_to_yolo import visual_genome_to_yolo_data
from source.visual_genome_to_yolo import visual_genome_to_yolo_data_n
from source.visual_genome_meta_data import plot_image_with_multiple_bboxes
from source.visual_genome_meta_data import get_image_ids
from source.yolo_training_structure import distribute_train_val_files as dist_train_val


## Test the visual genome to yolo workflow
#### 1. Load meta data
#### 2. Make class map for desired object class (mapping object class name to number)
#### 3. Save class map in yolo compatible format
#### 4. Reload meta data from yolo compatible class map file
#### 5. Save meta data about desired object class in yolo compatible meta data files (for images without desired object class the file is empty)
#### 6. Plot images with desired object class with corresponding bounding boxes (based on class map and meta data files)
#### 7. Check if objects of desired object class have appropriate bounding boxes

### 1. Load data about desired object class

#### Define paths: 

In [ ]:
root_path = Path('/Users/stephanehess/Documents/CAS_AML/dias_digit_project')
data_path = root_path / 'test_visual_genome'
yolo_path = root_path / 'test_visual_genome_yolo'
file_path = data_path/'objects_entire.json'

#### Read in objects file with meta data about visual genome data: 

In [ ]:
objects = read_json_to_dict(file_path)

In [ ]:
len(objects)

In [ ]:
#objects

#### Get image identifiers: 

In [ ]:
image_id_list = get_image_ids(data_path)
image_id_list.sort()
len(image_id_list)

In [ ]:
image_id_list[0:3]

### 2. Put meta data about desired object class into class map

#### Choose the desired object class:

In [ ]:

desired_objects = ['man']

desired_objects

#### Create class map based on desired objects: 

In [ ]:
class_map = create_class_mapping_from_list(desired_objects)

In [ ]:
class_map

### 3. Save class map in yolo compatible format

In [ ]:

file_path = str(yolo_path) + '/'

output_path = file_path + 'class_map.yaml'
save_class_map_to_yaml(class_map, output_path)

### 4. Reload meta data from yolo compatible class map file

In [ ]:
file_list = os.listdir(yolo_path)
for filename in file_list:
    if filename.split('_')[-1] == 'map.yaml':
        yaml_file_name = filename
yaml_file_name

In [ ]:
yaml_path = yolo_path/yaml_file_name

class_map = read_yaml_to_class_map(str(yaml_path))


#### Check content of class_map.yaml file:

In [ ]:

# Print the class mapping
print(class_map)

In [ ]:
#convert_single_image_to_yolo(objects[0], class_map, data_path, yolo_path)

### 5. Save meta data about desired object class in yolo compatible meta data files

#### The yolo compatible meta data files have to contain bounding box information for images containing the desired object:

In [ ]:
len(objects)

#### Prepare input for visual_genome_to_yolo_data function:

In [ ]:
objects_and_ids = (objects, desired_objects, image_id_list)
# objects: contains meta data about all visual genome images.
# desired_objects: list of desired object classes
# image_id_list: list of ids designating selection of images containing the desired object class
# among the images in the test_visual_genome directory (i.e. not all the visual genome images with the
# desired object class).

paths = (data_path, yolo_path)
# data_path: path to directory containing images.
# yolo_path: path to directory where the yolo compatible meta data files are going to be saved.

In [ ]:

objects_and_ids = (objects, desired_objects, image_id_list)
paths = (data_path, yolo_path)

#label_paths_w, occurrence_counts = visual_genome_to_yolo_data(objects_and_ids, paths, class_map)
#len(label_paths_w)

label_paths_w, occurrence_counts = visual_genome_to_yolo_data_n(objects_and_ids, paths, class_map)
len(label_paths_w)

In [ ]:
occurrence_counts

In [ ]:
label_paths_w[0:3]

In [ ]:
class_map

In [ ]:
occurrence_counts

#### Get number of required images without desired object for balanced data set:

In [ ]:
desired_objects

In [ ]:
desired_objects[0]

In [ ]:
if len(desired_objects) == 1:
    number_occurrences = occurrence_counts[desired_objects[0]]
    print(number_occurrences)
else:
    number_occurrences = 'No unique answer: more than one desired objects!'
    print(number_occurrences)

In [ ]:
number_images_without = round((number_occurrences/100) * 30)
number_images_without

#### Create meta data text files for images without desired object:

In [ ]:
label_paths_n, occurrence_counts = visual_genome_to_yolo_data_n(objects_and_ids, paths, class_map,
                                                           with_class = False, number_of_images = number_images_without)
len(label_paths_n)

In [ ]:
label_paths_n[0:3]

#### Have a look at the numbers of files with and without desired object:

In [ ]:
# Paths to label files with desired objects:
print(type(label_paths_w))
print(len(label_paths_w))
# Paths to label files without desired objects: 
print(type(label_paths_n))
print(len(label_paths_n))

In [ ]:
label_paths = label_paths_w + label_paths_n
len(label_paths)

### 6. Plot selected images (with and without desired object class) and use class_map file to plot bounding boxes:

In [ ]:
for label_path in label_paths:
    print(label_path)
    img_id = label_path.split('_')[-1].split('.')[0]
    print(img_id)
    
    labels, bboxes = read_yolo_metadata(label_path, class_map)
    class_names = list(labels)
    image_path_gen = data_path/'visual_genome_'
    image_path = str(image_path_gen) + str(img_id) + '.jpg'
    print(image_path)
    plot_image_with_multiple_bboxes(image_path, bboxes, class_names)
    

### 7. The two images containing the desired object class were correctly selected, one image without desired object class was correctly selected, bounding boxes were correctly placed around desired objects.

## Test conversion to yolo compatible structure for one single image

In [ ]:
len(objects[0]['objects'])

In [ ]:
obj_list = [objects[0]['objects'][idx]['names'][0] for idx in range(0, len(objects[0]['objects']))]
obj_list[0:3]

In [ ]:
len(obj_list)

In [ ]:
obj_list.sort()
obj_list

In [ ]:
#desired_objects = obj_list[0:7]
desired_objects = ['shirt']

In [ ]:
obj_list.count('shirt')

In [ ]:
class_map = create_class_mapping_from_list(desired_objects)

In [ ]:
class_map

In [ ]:
image_dir = data_path/'convert_single_img_test'
output_dir = yolo_path/'convert_single_img_test'

In [ ]:
label_path = convert_single_image_to_yolo(objects[0], class_map, image_dir, output_dir)

In [ ]:
label_paths = [label_path]

In [ ]:
for label_path in label_paths:
    print(label_path)
    img_id = label_path.split('_')[-1].split('.')[0]
    print(img_id)
    
    labels, bboxes = read_yolo_metadata(label_path, class_map)
    class_names = list(labels)
    image_path_gen = data_path/'visual_genome_'
    image_path = str(image_path_gen) + str(img_id) + '.jpg'
    print(image_path)
    plot_image_with_multiple_bboxes(image_path, bboxes, class_names)

In [ ]:
file_path = output_dir/'visual_genome_1.txt'

with open(file_path, 'r') as f:
   lines = f.readlines()

In [ ]:
lines

In [ ]:
type(lines)

In [ ]:
len(lines)